In [1]:
from pytdx.hq import TdxHq_API
from pytdx.params import TDXParams
from datetime import datetime, timedelta
import pandas as pd
import polars as pl
import os
from tqdm import tqdm

HOST = '119.147.212.81'
PORT = 7709

API_TICK_SIZE = 2000

In [2]:
api = TdxHq_API()

In [3]:
df_cn_stks = pl.read_csv('stk_list/cn_list.csv')
df_cn_stks.head()

df_cn_stks['code'] = df_cn_stks['code'].apply(lambda x: str(x).zfill(6))
df_cn_stks.head()

symbol,name,tdx_api_mkt,code
str,str,i64,str
"""SZ002875""","""安奈儿""",0,"""002875"""
"""SZ002873""","""新天药业""",0,"""002873"""
"""SZ002870""","""香山股份""",0,"""002870"""
"""SZ002869""","""金溢科技""",0,"""002869"""
"""SZ002868""","""绿康生化""",0,"""002868"""


In [4]:
with api.connect(HOST, PORT):
    for i in tqdm(list(range(len(df_cn_stks[0:])))):
        mkt = df_cn_stks['tdx_api_mkt'][i]
        code = df_cn_stks['code'][i]
        
        # # manually set symbol
        # mkt = 1
        # code = '688076'

        data = api.get_security_bars(5, mkt, code, 1, 150)
        df_tmp = api.to_df(data)

        df = df_tmp[['open', 'close', 'high', 'low', 'vol', 'amount', 'datetime']]
        df = pl.DataFrame(df)
        df.to_parquet(f'cn_data_weekly_bar/data/{code}.parquet')

        # break

100%|██████████| 2024/2024 [02:49<00:00, 11.91it/s]
